In [7]:
from typing import List
import quartz # type: ignore
import time
import dgl

quartz_context = None

def init_quartz_context(
    gate_set: List[str] = ['h', 'cx', 'x', 'rz', 'add'],
    ecc_file_path: str = '../ecc_set/nam_ecc.json',
    no_increase: bool = False,
    include_nop: bool = True,
):
    global quartz_context
    quartz_context = quartz.QuartzContext(
        gate_set=gate_set,
        filename=ecc_file_path,
        no_increase=no_increase,
        include_nop=include_nop,
    )
    return quartz_context

def qasm_to_graph(qasm_str: str) -> quartz.PyGraph:
    graph = quartz.PyGraph.from_qasm_str(context=quartz_context, qasm_str=qasm_str)
    return graph

def file_to_graph(filename: str) -> quartz.PyGraph:
    with open(filename, 'r') as f:
        qasm_str = f.read()
    return qasm_to_graph(qasm_str)

def is_nop(xfer_id: int) -> bool:
    return quartz_context.get_xfer_from_id(id=xfer_id).is_nop

In [5]:
init_quartz_context(
    gate_set=['h', 'cz', 'rz', 'rx1', 'rx3', 'x', 'add'],
    ecc_file_path='../ecc_set/rigetti_5_ecc.json',
    no_increase=False,
    include_nop=True,
)
qasm_file = '../rigetti_circs/twolocalrandom_nativegates_rigetti_qiskit_opt0_10_norm.qasm'
graph = file_to_graph(qasm_file)
print(graph.gate_count)

1105


In [13]:
init_quartz_context(
    gate_set=['h', 'cx', 'x', 'rz', 'add'],
    ecc_file_path='../ecc_set/nam_ecc.json',
    no_increase=False,
    include_nop=True,
)
qasm_file = '../nam_circs/mod_adder_1024.qasm'
graph = file_to_graph(qasm_file)
print(graph.gate_count)

g1 = graph.to_dgl_graph()
subg, new_indices = dgl.khop_out_subgraph(g1, 10, k=6)
print(f'{subg = }\n{new_indices = }\n{subg.ndata["gate_type"].shape = }')
print(f'{new_indices.shape = }')

4285
subg = Graph(num_nodes=23, num_edges=56,
      ndata_schemes={'gate_type': Scheme(shape=(), dtype=torch.int32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'src_idx': Scheme(shape=(), dtype=torch.int32), 'dst_idx': Scheme(shape=(), dtype=torch.int32), 'reversed': Scheme(shape=(), dtype=torch.int32), '_ID': Scheme(shape=(), dtype=torch.int64)})
new_indices = tensor([1])
subg.ndata["gate_type"].shape = torch.Size([23])
new_indices.shape = torch.Size([1])
